# Web Scrape Private ESPN Fantasy Football League <a id="return"></a>

This notebook will scrape a private ESPN Fantasy Football league and return league/roster data.
<br><br/>

**Notebook Sections:**
1. [Import Packages and Set User-Defined Fields](#section1)
2. [Web Scrape Private ESPN Fantasy Football League and Load Data ](#section2)
3. [Create Rosters Dataframes](#section3)
4. [Create Matchups Dataframes](#section4)

**Outputs:**
1. Web-Scraped Weekly Matchup Raw Data: **json**
2. Weekly Roster Data: **csv**
3. Weekly Roster Data with Player/Fantasy Stats: **csv**
4. Weekly Matchup Data: **csv**
5. Total Wins/Losses by Fantasy Team: **csv**

* https://stmorse.github.io/journal/espn-fantasy-3-python.html
* https://pypi.org/project/power-ranker/
* https://www.youtube.com/watch?v=PUUkwBYYKHA

## Import Packages <a id="section1"></a>

In [1]:
# increase cell width of this notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [1]:
print('test')

test


In [2]:
# import needed packages
import pandas as pd

# set pandas display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

# run the 00-scrape_espn_ff_api_v3_util.ipynb notebook
%run 00-scrape_espn_ff_api_v3_util.ipynb

## Set User-Defined Fields

In order to web scrape data from a private ESPN Fantasy Football League, we need to pass the SWID and the espn_s2 cookies values to the cookies parameter of the requests.get command.
<br><br/>

In Chrome the SWID and the espn_s2 cookies values can be found here, chrome://settings/cookies/detail?site=espn.com.
<br><br/>

To manually find the cookies values follow these instructions: Settings > Privacy and security > Cookies and other site data > All cookies and site data > espn.com > SWID/espn_s2.

<img src="pictures/settings_cookies.PNG">
<br><br/>

Please note the cookies values differ from machine to machine.  My cookies values won't work on another comptuer.

<img src="pictures/SWID_cookie.PNG">

<img src="pictures/espn_s2_cookie.PNG">
<br><br/>

Additionally, we need to pass the league id into the url.
<br><br/>

To find the league id look in the url while logged into a private ESPN Fantasy Football league.

<img src="pictures/league_id.PNG">
<br><br/>

Finally, to review any matchup:
<br><br/>

Set teamId to the team id value and seasonId to 2021, 2020, 2019, or 2018 in the web URL.  ESPN's API v3 doesn't return data for years prior to 2018.
<br><br/>

If the matchup under review is a **non-playoff matchup** then set both matchupPeriodId and scoringPeriodId to the same week number value.
<br><br/>

If the matchup under review is a **playoff matchup** then the matchupPeriodId and scoringPeriodId week values are different.  For example, if we're reviewing a matchup during NFL weeks 15 and 16, which correspond to the last 2 playoff games in our league, then matchupPeriodId = 14 and scoringPeriodId = 15 or 16 for NFL weeks 15 or 16.
<br><br/>

For example, https://fantasy.espn.com/football/boxscore?leagueId=169073&matchupPeriodId=1&scoringPeriodId=1&seasonId=2020&teamId=10&view=scoringperiod
<br><br/>

Source: https://stmorse.github.io/journal/espn-fantasy-v3.html
<br><br/>

[Return to Top](#return)

In [3]:
# set user-defined fields below: cookie values, league id, season, and current NFL week number
swid      = '{F3E3A4D2-1203-4B11-BDAD-00C92AAA3F48}'
espn_s2   = 'AECf2TISP%2BRluNYIj24%2FqkuJBHTzAa7Br0BVIoUnBhHQCygnkN7hvyvU8wIm6XAT58otHnOZW4HjyImLJ14Rk8L9%2BwObWXeIa8kCosNFNMSc79r24KSiJK9jtqUQ2V1lvIncFM1qDhV9xL7E5jnutCP9rZfiJv8h%2Bq6WNvFb4YmyHICgx3QW68obk3wqBrGrXmw0vbq2bf367%2BsuL%2BOJ2ioRnxi1yY7LgbtiJnAXSKZj7kdIGjBiCPjUHgeXzDLHPunkLchEGlOrr%2FhG1ORgOsUX'
league_id = 169073
season    = 2021
week      = 17

# set flag to decide whether or not to web scrape ESPN fantasy football league
scrape_espn = False

# set flag to decide whether or not to combine rosters from multiple seasons into a single dataframe and save it to csv
combine_years = False

## Web Scrape Private ESPN Fantasy Football League and Load Data <a id="section2"></a>

In this section, we'll scrape a private ESPN Fantasy Football league and store each week's matchups data in a json file as well as in a list variable.
<br><br/>

**NOTE:** Only need to scape ESPN once, after which the scrape_espn flag should be set to False.
<br><br/>

[Return to Top](#return)

In [4]:
%%time

# instantiate data_ingest object
data_ingest = data_ingest(swid, espn_s2, league_id, season, week)

# check scrape_espn flag
if scrape_espn:
    
    #scrape ESPN FF league and save to json
    data_ingest.scrape_espn_data()

# load json files with each's week matchup data
matchups_list, data = data_ingest.load_data_from_disk()

Wall time: 2 s


## Create Rosters Dataframes <a id="section3"></a>

In this section, we'll create dataframes consisting of...

1. each fantasy football teams' weekly rosters and save it to a csv file
2. each fantasy football teams' weekly rosters (including player stats and fantasy scoring stats) and save it to a csv file.
<br><br/>

[Return to Top](#return)

In [5]:
%%time

# instantiate create_rosters object
create_rosters = create_rosters(matchups_list, season)

# create dataframe of fantasy football teams' weekly rosters and save it to a csv file
rosters_df = create_rosters.create_weekly_rosters()

# create dataframe of fantasy football teams' weekly rosters (including player stats and fantasy scoring stats) and save it to a csv file
rosters_df_w_scoring = create_rosters.create_weekly_rosters_w_scoring(rosters_df)

Weekly Rosters Shape: (3400, 13)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                3400 non-null   int64  
 1   week                3400 non-null   int64  
 2   owner_team          3400 non-null   object 
 3   owner               3400 non-null   object 
 4   player              3400 non-null   object 
 5   pro_team            3400 non-null   object 
 6   pro_team_abv        3390 non-null   object 
 7   current_inj_status  3128 non-null   object 
 8   lineup_slot_name    3400 non-null   object 
 9   position_name       3400 non-null   object 
 10  proj_points         3382 non-null   float64
 11  actual_points       3210 non-null   float64
 12  slot_id             3400 non-null   int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 345.4+ KB


None

,year,week,proj_points,actual_points,slot_id
count,3400.0,3400.0000,3382.000000,3210.000000,3400.000000
mean,2021.0,9.0000,11.615468,12.006417,13.904118
std,0.0,4.8997,7.671807,10.679309,8.068564
min,2021.0,1.0000,0.000000,-6.500000,0.000000
25%,2021.0,5.0000,7.211607,3.800000,4.000000
50%,2021.0,9.0000,11.082216,10.000000,20.000000
75%,2021.0,13.0000,16.374103,17.800000,20.000000
max,2021.0,17.0000,36.933048,71.800000,23.000000


,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id
0,2021,1,Happy Rock Homewreckers,Blainer,Austin Ekeler,Los Angeles Chargers,LAC,ACTIVE,RB,RB,16.812705,12.6,2
1,2021,1,Happy Rock Homewreckers,Blainer,Saquon Barkley,New York Giants,NYG,ACTIVE,RB,RB,15.478928,4.0,2
2,2021,1,Happy Rock Homewreckers,Blainer,CeeDee Lamb,Dallas Cowboys,DAL,ACTIVE,WR,WR,16.326247,28.0,4
3,2021,1,Happy Rock Homewreckers,Blainer,Mike Evans,Tampa Bay Buccaneers,TB,ACTIVE,WR,WR,16.669912,5.4,4
4,2021,1,Happy Rock Homewreckers,Blainer,Mark Andrews,Baltimore Ravens,BAL,ACTIVE,TE,TE,12.950163,5.4,6


Weekly Rosters Shape w/ Scoring: (3400, 236)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Columns: 236 entries, year to unk210
dtypes: float64(225), int64(3), object(8)
memory usage: 6.1+ MB


None

,year,week,proj_points,actual_points,slot_id,pass_comp_ff,pass_incomp_ff,pass_td_ff,pass_5_yrd_ff,pass_50_yrd_td_ff,pass_yrd_300_399_ff,pass_yrd_400+_ff,pass_2pt_con_ff,pass_int_ff,rush_td_ff,rush_2pt_con_ff,rush_5_yrd_ff,rush_50_yrd_td_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rec_td_ff,rec_2pt_con_ff_ff,rec_50_yrd_td_ff,rec_5_yrd_ff,receptions_ff,rec_yrd_100_199_ff,rec_yrd_200+_ff,fum_lost_ff,fg_made_40_49_ff,fg_miss_40_49_ff,fg_made_0_39_ff,fg_miss_0_39_ff,pat_made_ff,pat_miss_ff,def_st_0_pts_alw_ff,def_st_1_6_pts_alw_ff,def_st_7_13_pts_alw_ff,def_st_14_17_pts_alw_ff,def_st_blk_td_ff,def_st_int_ff,def_st_fum_ff,def_st_blk_kick_ff,def_st_safety_ff,def_st_sack_ff,def_st_kick_ret_td_ff,def_st_punt_ret_td_ff,def_st_int_td_ff,def_st_fum_ret_td_ff,def_st_22_27_pts_alw_ff,def_st_28_34_pts_alw_ff,...,unk118,unk119,def_pts_alw,unk121,def_st_22_27_pts_alw,def_st_28_34_pts_alw,def_st_35_45_pts_alw,def_st_46+_pts_alw,def_tot_yrd_alw,def_st_0_99_yrd_alw,def_st_100_199_yrd_alw,def_st_200_299_yrd_alw,unk131,def_st_350_399_yrd_alw,def_st_400_449_yrd_alw,def_st_450_499_yrd_alw,def_st_500_549_yrd_alw,def_st_550+_yrd_alw,unk155,unk156,unk158,unk175,unk176,unk177,unk178,unk179,unk180,unk181,unk182,unk183,unk184,unk185,unk186,unk187,unk188,unk189,unk190,unk191,unk192,unk193,unk194,unk195,unk196,unk197,fg_made_50_59,unk199,unk200,unk202,unk203,unk210
count,3400.0,3400.0000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.0,3400.000000,3400.000000,3400.000000,3400.000000,...,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.00000,3400.000000,3400.0,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.00000,3400.000000,3400.0,3400.0,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000
mean,2021.0,9.0000,11.553975,11.335471,13.904118,0.804941,-0.209529,0.924706,0.450353,0.036176,0.058235,0.017647,0.015294,-0.144706,0.670588,0.008235,1.566882,0.010588,0.076765,0.001471,0.885882,0.016471,0.033529,2.782235,2.198529,0.132353,0.004412,-0.079412,0.084706,-0.004412,0.163235,-0.004706,0.123824,-0.006471,0.020588,0.026765,0.046765,0.011471,0.012353,0.206471,0.123529,0.011765,0.001176,0.187353,0.010588,0.0,0.030882,0.005882,-0.014706,-0.045000,...,0.118529,0.026765,1.475294,0.009118,0.014706,0.01500,0.002941,0.0,24.447059,0.000294,0.005294,0.020588,0.016765,0.018235,0.007353,0.004118,0.001176,0.000882,0.446176,0.386176,2.904118,0.072353,0.038235,0.017941,0.007059,0.087353,0.013529,0.005294,0.001176,0.065000,0.039706,0.017059,0.005588,1.475294,0.002059,0.003824,0.015588,0.011471,0.009118,0.014706,0.01500,0.002941,0.0,0.0,0.014412,0.020294,0.005882,0.001765,0.000882,0.834412
std,0.0,4.8997,7.697768,10.736864,8.068564,2.665756,0.709572,3.675124,1.501281,0.365692,0.413962,0.296563,0.180878,0.703207,2.281657,0.128092,3.265640,0.177938,0.473780,0.085749,2.499528,0.187171,0.323711,4.036892,2.652969,0.616160,0.148478,0.405377,0.680845,0.066284,0.910274,0.108379,0.634608,0.080191,0.453342,0.432078,0.371682,0.106500,0.272005,1.115440,0.753167,0.160473,0.048500,0.795629,0.251865,0.0,0.410138,0.171423,0.120391,0.364711,...,0.580965,0.189892,5.788514,0.095064,0.120391,0.12157,0.054161,0.0,89.302904,0.017150,0.072579,0.142022,0.128408,0.133821,0.08

,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id,pass_comp_ff,pass_incomp_ff,pass_td_ff,pass_5_yrd_ff,pass_50_yrd_td_ff,pass_yrd_300_399_ff,pass_yrd_400+_ff,pass_2pt_con_ff,pass_int_ff,rush_td_ff,rush_2pt_con_ff,rush_5_yrd_ff,rush_50_yrd_td_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rec_td_ff,rec_2pt_con_ff_ff,rec_50_yrd_td_ff,rec_5_yrd_ff,receptions_ff,rec_yrd_100_199_ff,rec_yrd_200+_ff,fum_lost_ff,fg_made_40_49_ff,fg_miss_40_49_ff,fg_made_0_39_ff,fg_miss_0_39_ff,pat_made_ff,pat_miss_ff,def_st_0_pts_alw_ff,def_st_1_6_pts_alw_ff,def_st_7_13_pts_alw_ff,def_st_14_17_pts_alw_ff,def_st_blk_td_ff,def_st_int_ff,def_st_fum_ff,def_st_blk_kick_ff,...,unk118,unk119,def_pts_alw,unk121,def_st_22_27_pts_alw,def_st_28_34_pts_alw,def_st_35_45_pts_alw,def_st_46+_pts_alw,def_tot_yrd_alw,def_st_0_99_yrd_alw,def_st_100_199_yrd_alw,def_st_200_299_yrd_alw,unk131,def_st_350_399_yrd_alw,def_st_400_449_yrd_alw,def_st_450_499_yrd_alw,def_st_500_549_yrd_alw,def_st_550+_yrd_alw,unk155,unk156,unk158,unk175,unk176,unk177,unk178,unk179,unk180,unk181,unk182,unk183,unk184,unk185,unk186,unk187,unk188,unk189,unk190,unk191,unk192,unk193,unk194,unk195,unk196,unk197,fg_made_50_59,unk199,unk200,unk202,unk203,unk210
0,2021,1,Happy Rock Homewreckers,Blainer,Austin Ekeler,Los Angeles Chargers,LAC,ACTIVE,RB,RB,16.812705,12.6,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2021,1,Happy Rock Homewreckers,Blainer,Saquon Barkley,New York Giants,NYG,ACTIVE,RB,RB,15.478928,4.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2021,1,Happy Rock Homewreckers,Blainer,CeeDee Lamb,Dallas Cowboys,DAL,ACTIVE,WR,WR,16.326247,28.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,12.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2021,1,Happy Rock Homewreckers,Blainer,Mike Evans,Tampa Bay Buccaneers,TB,ACTIVE,WR,WR,16.669912,5.4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2021,1,Happy Rock Homewreckers,Blainer,Mark Andrews,Baltimore Ravens,BAL,ACTIVE,TE,TE,12.950163,5.4,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Wall time: 1min 16s


In [6]:
# combine rosters from multiple seasons into single dataframe
if combine_years:
    
    if season == 2021:

        rosters_df_2021 = rosters_df.copy()
        rosters_df_w_scoring_2021 = rosters_df_w_scoring.copy()
        
    if season == 2020:

        rosters_df_2020 = rosters_df.copy()
        rosters_df_w_scoring_2020 = rosters_df_w_scoring.copy()
        
    if season == 2019:

        rosters_df_2019 = rosters_df.copy()
        rosters_df_w_scoring_2019 = rosters_df_w_scoring.copy()
        
    if season == 2018:

        rosters_df_2018 = rosters_df.copy()
        rosters_df_w_scoring_2018 = rosters_df_w_scoring.copy()
        
    try:
        rosters_df_2018, rosters_df_2019, rosters_df_2020, rosters_df_2021
        
    except:
        print("haven't ran web scraper for all years yet...")
    
    else:
        
        rosters_df_all = pd.concat([rosters_df_2021, rosters_df_2020, rosters_df_2019, rosters_df_2018]).reset_index(drop=True)        
        rosters_df_all.to_pickle("../data/rosters_df_all.pkl")
        rosters_df_all.info()
        
        rosters_df_w_scoring_all = pd.concat([rosters_df_w_scoring_2021, rosters_df_w_scoring_2020, rosters_df_w_scoring_2019, rosters_df_w_scoring_2018]).reset_index(drop=True)
        rosters_df_w_scoring_all.to_pickle("../data/rosters_df_w_scoring_all.pkl")
        rosters_df_w_scoring_all.info()

## Create Matchups Dataframes <a id="section4"></a>

In this section, we'll create dataframes consisting of...

1. each fantasy football teams' weekly matchups and save it to a csv file
2. each fantasy football teams' total wins/losses through the most recent NFL week and save it to a csv file.
<br><br/>

[Return to Top](#return)

In [7]:
%%time

# instantiate create_rosters object
create_matchups = create_matchups(season)

# create dataframe of weekly matchups
matchups_df = create_matchups.create_weekly_matchups(data)

# create dataframe of total wins/losses
win_loss_df = create_matchups.create_wins_losses(matchups_df)

Weekly Matchups Shape: (168, 8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   week                 168 non-null    int64  
 1   owner_team_name      168 non-null    object 
 2   owner                168 non-null    object 
 3   score                168 non-null    float64
 4   win                  168 non-null    int64  
 5   opp_owner_team_name  168 non-null    object 
 6   opp_owner            168 non-null    object 
 7   opp_score            168 non-null    float64
dtypes: float64(2), int64(2), object(4)
memory usage: 10.6+ KB


None

,week,score,win,opp_score
count,168.000000,168.000000,168.000000,168.000000
mean,8.928571,153.259524,0.500000,153.259524
std,4.898456,29.602017,0.501495,29.602017
min,1.000000,40.600000,0.000000,40.600000
25%,5.000000,132.525000,0.000000,132.525000
50%,9.000000,156.050000,0.500000,156.050000
75%,13.000000,173.650000,1.000000,173.650000
max,17.000000,228.900000,1.000000,228.900000


,week,owner_team_name,owner,score,win,opp_owner_team_name,opp_owner,opp_score
0,1,Sticky Icky,T-$,156.9,0,CoMo FightinCamlToes,Doisy,167.3
1,1,CoMo FightinCamlToes,Doisy,167.3,1,Sticky Icky,T-$,156.9
2,1,Springfield Atoms,Duvi,168.2,1,Happy Rock Homewreckers,Blainer,103.4
3,1,Happy Rock Homewreckers,Blainer,103.4,0,Springfield Atoms,Duvi,168.2
4,1,Pixel Whippers,Sembower,169.5,0,Seattle rainier riot,Boob,204.8


Total Wins/Losses Shape: (10, 5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   owner_team_name  10 non-null     object 
 1   wins             10 non-null     int32  
 2   losses           10 non-null     int32  
 3   points_for       10 non-null     float64
 4   points_against   10 non-null     float64
dtypes: float64(2), int32(2), object(1)
memory usage: 400.0+ bytes


None

,wins,losses,points_for,points_against
count,10.000000,10.000000,10.000000,10.000000
mean,8.400000,8.400000,2574.760000,2574.760000
std,1.577621,1.776388,182.963574,142.292088
min,6.000000,6.000000,2293.100000,2370.200000
25%,7.000000,7.250000,2471.800000,2490.900000
50%,9.000000,8.000000,2501.850000,2541.550000
75%,9.000000,10.000000,2763.050000,2703.525000
max,11.000000,11.000000,2804.800000,2751.300000


,owner_team_name,wins,losses,points_for,points_against
0,Beacon Hill Posterizers,9,8,2492.1,2549.0
1,Bench Don't Kill My Vibe,6,11,2293.1,2534.1
2,Brookside Shokunin,9,8,2706.8,2751.3
3,Bud Lathrop Drive,7,10,2781.8,2700.0
4,CoMo FightinCamlToes,11,6,2470.6,2489.2


Wall time: 109 ms


In [8]:
# combine matchups from multiple seasons into single dataframe
if combine_years:
    
    columns = ['year','week','owner_team_name','owner','score','win','opp_owner_team_name','opp_owner','opp_score']
    
    if season == 2021:

        matchups_df_2021 = matchups_df.copy()
        matchups_df_2021['year'] = 2021
        matchups_df_2021 = matchups_df_2021[columns].sort_values('week')
        matchups_df_2021.info()
        
    if season == 2020:

        matchups_df_2020 = matchups_df.copy()
        matchups_df_2020['year'] = 2020
        matchups_df_2020 = matchups_df_2020[columns].sort_values('week')
        matchups_df_2020.info()
        
    if season == 2019:

        matchups_df_2019 = matchups_df.copy()
        matchups_df_2019['year'] = 2019
        matchups_df_2019 = matchups_df_2019[columns].sort_values('week')
        matchups_df_2019.info()
        
    if season == 2018:

        matchups_df_2018 = matchups_df.copy()
        matchups_df_2018['year'] = 2018
        matchups_df_2018 = matchups_df_2018[columns].sort_values('week')
        matchups_df_2018.info()
        
    try:
        matchups_df_2018, matchups_df_2019, matchups_df_2020, matchups_df_2021
        
    except:
        print("\nhaven't ran web scraper for all years yet...")
    
    else:
        
        matchups_df_all = pd.concat([matchups_df_2021, matchups_df_2020, matchups_df_2019, matchups_df_2018]).reset_index(drop=True)
        matchups_df_all.to_pickle(f"../data/matchups_df_all.pkl")
        matchups_df_all.info()